In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from scipy.stats import ks_2samp
from scipy.stats import skew, kurtosis
import python_utils as utils
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense, Input, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
import random
import seaborn as sns
from tensorflow.keras.layers import LayerNormalization
import tensorflow_probability as tfp
from scipy.stats import t as student_t
from tensorflow.keras.layers import Dense
from tensorflow.keras import initializers
import textwrap
import tensorflow as tf
from tensorflow.keras import layers, initializers
import tensorflow as tf, os, datetime as dt
import tensorflow_probability as tfp
tfs = tfp.stats
from scipy.stats import wasserstein_distance
import os, pickle
from scipy.stats import wasserstein_distance
import os, shutil, glob, datetime as dt
from pathlib import Path

In [ ]:
file_path = "/content/SP500_data_fixed.xlsx"
xls = pd.ExcelFile(file_path)

print(xls.sheet_names)

df = pd.read_excel(xls, sheet_name=0, index_col=0, parse_dates=True)

print(df.head())


selected_stocks = ["META", "TROW"]

df = df[selected_stocks]

df = df.loc["2013-01-01":"2024-01-01"]

print(df.head())

# Save data
df.to_csv("meta_msft_2013_2019.csv")

csv_filename = "meta_msft_2013_2019.csv"
df.to_csv(csv_filename)
print(f"Data successfully saved to {csv_filename}")

In [ ]:
file_path = "/content/SP500_data_fixed.xlsx"
xls = pd.ExcelFile(file_path)

print(xls.sheet_names)

df = pd.read_excel(xls, sheet_name=0, index_col=0, parse_dates=True)
selected_stocks = ["META", "TROW"]
df = df[selected_stocks]
df = df.loc["2013-01-01":"2024-01-01"]

# Compute log returns (stationary transformation)
df_log = np.log(df).diff().dropna()

# Save the processed data
csv_filename = "meta_msft_logreturns.csv"
df_log.to_csv(csv_filename)
print(f"Data saved to {csv_filename}")

# Check
print(df_log.head())

In [ ]:
# Reset the index and drop the old index (Date column)
df_log.reset_index(drop=True, inplace=True)

# Now the DataFrame will be indexed by integers (0, 1, 2, ...)
print(df_log.head())

In [ ]:
plt.figure(figsize=(14, 6))
for stock in selected_stocks:
    plt.plot(df.index, df[stock], label=stock)

plt.xlabel("Date")
plt.ylabel("Price")
plt.title("Stock Price Sequences (2018-2019)")
plt.legend()
plt.show()

In [ ]:
# def load_and_preprocess(filename, seq_len):
#     """
#     Load your CSV data, sort it in chronological order,
#     and segment it into overlapping sequences.

#     Args:
#       - filename: path to the CSV file.
#       - seq_len: the desired sequence length.

#     Returns:
#       - sequences: a NumPy array of shape (number_of_sequences, seq_len, num_features)
#     """
#     file_path = "/content/meta_msft_2013_2019.csv"
#     # Load the CSV file (ensure the index is the date)
#     df = pd.read_csv(filename, index_col=0, parse_dates=True)
#     # Sort by the date to ensure chronological order
#     df = df.sort_index()

#     # Convert DataFrame to a NumPy array.
#     data = df.values  # shape: (num_days, num_features)

#     # Create overlapping sequences.
#     sequences = []
#     for i in range(0, len(data) - seq_len + 1):
#         sequences.append(data[i:i + seq_len])
#     sequences = np.array(sequences)
#     return sequences

# # Example usage:
# csv_file = "/content/meta_msft_2013_2019.csv"
# seq_len = 260  # For example, each sequence is 15 days long.
# ori_data = load_and_preprocess(csv_file, seq_len)
# print("Shape of ori_data:", ori_data.shape)

In [ ]:
def load_and_preprocess(filename, seq_len):
    df = pd.read_csv(filename, index_col=0, parse_dates=True)
    df = df.sort_index()
    data = df.values

    sequences = []
    for i in range(len(data) - seq_len + 1):
        sequences.append(data[i:i + seq_len])
    sequences = np.array(sequences)
    return sequences

# Usage example
csv_file = "meta_msft_logreturns.csv"
seq_len = 120 # was 60 in the best attempt
ori_data = load_and_preprocess(csv_file, seq_len)
print("Shape of ori_data (log returns):", ori_data.shape)

In [ ]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.85
config.gpu_options.visible_device_list = "0"
session = InteractiveSession(config=config)

# Force TF to use non-CuDNN implementation
tf.config.experimental.enable_op_determinism()

In [ ]:
### Utility Functions ###
def train_test_divide (data_x, data_x_hat, data_t, data_t_hat, train_rate = 0.8):
  """Divide train and test data for both original and synthetic data.

  Args:
    - data_x: original data
    - data_x_hat: generated data
    - data_t: original time
    - data_t_hat: generated time
    - train_rate: ratio of training data from the original data
  """
  # Divide train/test index (original data)
  no = len(data_x)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x = [data_x[i] for i in train_idx]
  test_x = [data_x[i] for i in test_idx]
  train_t = [data_t[i] for i in train_idx]
  test_t = [data_t[i] for i in test_idx]

  # Divide train/test index (synthetic data)
  no = len(data_x_hat)
  idx = np.random.permutation(no)
  train_idx = idx[:int(no*train_rate)]
  test_idx = idx[int(no*train_rate):]

  train_x_hat = [data_x_hat[i] for i in train_idx]
  test_x_hat = [data_x_hat[i] for i in test_idx]
  train_t_hat = [data_t_hat[i] for i in train_idx]
  test_t_hat = [data_t_hat[i] for i in test_idx]

  return train_x, train_x_hat, test_x, test_x_hat, train_t, train_t_hat, test_t, test_t_hat

def extract_time(data):
    """Returns Maximum sequence length and each sequence length.

  Args:
    - data: original data

  Returns:
    - time: extracted time information
    - max_seq_len: maximum sequence length
  """
    time = [len(seq) for seq in data]
    max_seq_len = max(time)
    return time, max_seq_len

def rnn_cell(module_name, hidden_dim):
    assert module_name in ['gru', 'lstm']
    if module_name == 'gru':
        return tf.keras.layers.GRUCell(hidden_dim)
    elif module_name == 'lstm':
        return tf.keras.layers.LSTMCell(hidden_dim)


# def masked_mse(y_true, y_pred, seq_lens):
#     """Properly handles variable-length sequences"""
#     # Create mask from sequence lengths
#     mask = tf.sequence_mask(
#         seq_lens,
#         maxlen=tf.shape(y_true)[1],
#         dtype=tf.float32
#     )

    # # Expand mask to match feature dimensions
    # mask = tf.expand_dims(mask, -1)  # [batch_size, seq_len, 1]

    # # Calculate masked MSE
    # squared_error = tf.square(y_true - y_pred)
    # masked_error = squared_error * mask

    # return tf.reduce_sum(masked_error) / (tf.reduce_sum(mask) + 1e-8)


def t_noise(shape, df=5, scale=0.6):
    """t(df) с единичной дисперсией * scale (0.5-0.7 обычно ок)."""
    sigma = np.sqrt(df/(df-2))        # σ исходного t
    return scale * student_t.rvs(df, size=shape) / sigma


def random_generator(batch_size, z_dim, T_mb, max_seq_len):
    """Generates a batch of random noise sequences."""
    Z_mb = np.zeros((batch_size, max_seq_len, z_dim))
    for i in range(min(batch_size, len(T_mb))):
        seq_length = T_mb[i]
        # Z_mb[i, :seq_length, :] = t.rvs(df=3, size=(seq_length, z_dim))
        Z_mb[i, :seq_length, :] = t_noise((seq_length, z_dim), df=5, scale=0.6)
        # Z_mb[i, :seq_length, :] = np.random.uniform(0., 1., (seq_length, z_dim))
    return Z_mb


def batch_generator(data, time, rolling_mu, rolling_sigma, crisis_intensities, batch_size):
    """Modified batch generator that includes crisis intensities"""
    idx = np.random.permutation(len(data))[:batch_size]
    seq_lens = [time[i] for i in idx]
    max_len = max(seq_lens)

    X_mb = np.array([np.pad(seq[:l], ((0,max_len-l),(0,0)), mode='edge')
                     for l, seq in zip(seq_lens, data[idx])])
    mu_mb = np.array([np.pad(rolling_mu[i][:l], ((0,max_len-l),(0,0)), 'edge')
                      for i,l in zip(idx, seq_lens)])
    sigma_mb = np.array([np.pad(rolling_sigma[i][:l], ((0,max_len-l),(0,0)), 'edge')
                         for i,l in zip(idx, seq_lens)])

    # Add crisis intensities
    crisis_mb = crisis_intensities[idx].reshape(-1, 1)  # [batch_size, 1]

    return X_mb, seq_lens, mu_mb, sigma_mb, crisis_mb




def exp_decay(initial, final, start_step, end_step, step):
    """
    Exponentially decays a weight from `initial` to `final`
    between [start_step, end_step].  Outside that window it is
    clamped to the respective edge value.
    """
    step = tf.cast(step, tf.float32)
    start_step = tf.cast(start_step, tf.float32)
    end_step   = tf.cast(end_step,   tf.float32)

    progress = tf.clip_by_value((step - start_step) / (end_step - start_step), 0.0, 1.0)
    log_interp = tf.math.log(initial) + progress * (tf.math.log(final) - tf.math.log(initial))
    return tf.exp(log_interp)

In [ ]:
class SNWrapper(tf.keras.layers.Wrapper):
    """
    Spectral Normalisation for every weight matrix in a layer,
    including recurrent_kernel for RNN cells.
    """
    def __init__(self, layer, power_iter: int = 1, **kwargs):
        super().__init__(layer, **kwargs)
        self.power_iter = power_iter
        self.w_refs, self.w_shapes, self.u_vecs = [], [], []
        self.current_sigma = None

    # -------------------------------------------------------------- #
    # locate kernels
    # -------------------------------------------------------------- #
    def build(self, input_shape):
        super().build(input_shape)

        def _add_kernel(w):
            if w is not None:
                self.w_refs.append(w)
                self.w_shapes.append(w.shape)
                self.u_vecs.append(
                    self.add_weight(
                        shape=(1, w.shape[-1]),
                        initializer="random_normal",
                        trainable=False,
                        name=f"sn_u_{len(self.u_vecs)}",
                    )
                )

        _add_kernel(getattr(self.layer, "kernel", None))

        cell = getattr(self.layer, "cell", None)
        if cell is not None:                       # LSTM/GRU wrapper
            _add_kernel(getattr(cell, "kernel", None))
            _add_kernel(getattr(cell, "recurrent_kernel", None))
        else:                                      # bare RNN cell
            _add_kernel(getattr(self.layer, "recurrent_kernel", None))

        if not self.w_refs:
            raise ValueError(f"No kernel found in layer {self.layer.name}")

    # -------------------------------------------------------------- #
    # spectral norm for a single matrix
    # -------------------------------------------------------------- #
    @staticmethod
    def _spectral_norm(w, u_var, power_iter):
        w_mat = tf.reshape(w, [-1, w.shape[-1]])
        u_hat = u_var
        for _ in range(power_iter):
            v_hat = tf.linalg.l2_normalize(tf.matmul(u_hat, w_mat, transpose_b=True))
            u_hat = tf.linalg.l2_normalize(tf.matmul(v_hat, w_mat))
        sigma = tf.matmul(tf.matmul(v_hat, w_mat), u_hat, transpose_b=True)
        u_var.assign(u_hat)
        return tf.reshape(w_mat / sigma, w.shape)

    # -------------------------------------------------------------- #
    # forward pass
    # -------------------------------------------------------------- #
    def call(self, inputs, training=None, **kwargs):
        originals = [tf.identity(w) for w in self.w_refs]   # <— fixed here

        for w, u in zip(self.w_refs, self.u_vecs):
            w.assign(self._spectral_norm(w, u, self.power_iter))

        out = self.layer(inputs, training=training, **kwargs)

        for w, w_orig in zip(self.w_refs, originals):
            w.assign(w_orig)

        return out

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)


In [ ]:
dim = ori_data.shape[2]  # Extract the number of features (columns)
max_seq_len = seq_len
z_dim = 8 # Latent space dimension

In [ ]:
parameters = {
    'hidden_dim': 64, # Number of hidden units in the RNN layers, was 64 in the best attempt
    'num_layer': 3, # Number of RNN layers
    'iterations': 20000, # Number of training iterations
    'batch_size': 64, # Size of each batch during training !!!!
    'z_dim': 10, # Latent space dimension
    'module': 'gru' # Define the RNN module ('lstm')
}

In [ ]:
# Network Parameters
hidden_dim = parameters['hidden_dim']
num_layers = parameters['num_layer']
iterations = parameters['iterations']
batch_size = parameters['batch_size']
module_name = parameters['module']
z_dim = parameters['z_dim']
gamma = 1

In [ ]:
class VolatilityScaler(tf.keras.layers.Layer):
    def __init__(self,
                 z_dim: int,
                 init_scale: float = 1.2,
                 init_sig: tf.Tensor | None = None):
        """
        Parameters
        ----------
        z_dim      : latent-feature dimensionality
        init_scale : fallback scale if `init_sig` is None
        init_sig   : 1-D tensor of length z_dim with the target
                     per-feature σ of real returns (may be NumPy
                     array – it is converted to tf.constant)
        """
        super().__init__()
        self.z_dim      = z_dim
        self.init_scale = init_scale
        self.init_sig   = (tf.constant(init_sig, dtype=tf.float32)
                           if init_sig is not None else None)

    def build(self, input_shape):
        assert len(input_shape) == 3 and input_shape[-1] == self.z_dim, \
            f"expected shape (batch, time, {self.z_dim})"

        if self.init_sig is not None:
            # Convert to numpy before using in initializer
            init_value = tf.math.log(self.init_sig)[None, None, :].numpy()
        else:
            # For scalar fallback, create numpy array directly
            init_value = tf.fill([1, 1, self.z_dim],
                              tf.math.log(self.init_scale)).numpy()

        self.log_scale = self.add_weight(
            name="log_scale",
            shape=(1, 1, self.z_dim),
            initializer=tf.constant_initializer(init_value),
            trainable=True,
        )

    def call(self, z):
        return z * tf.exp(self.log_scale)

In [ ]:
class SelfAttentionPooling(tf.keras.layers.Layer):
    def __init__(self, input_dim, **kwargs):
        super(SelfAttentionPooling, self).__init__(**kwargs)
        self.W = Dense(1, use_bias=False)

    def call(self, batch_rep):
        """
        Args:
            batch_rep: (batch_size, seq_len, hidden_dim)
        Returns:
            pooled_rep: (batch_size, hidden_dim)
        """
        # Compute attention weights
        att_scores = self.W(batch_rep)                    # (B,T,1)
        att_weights = tf.nn.softmax(att_scores, axis=1)   # (B,T,1)

        # Apply attention weights
        pooled_rep = tf.reduce_sum(batch_rep * att_weights, axis=1)  # (B,H)
        return pooled_rep

In [ ]:
# !pip install arch

In [ ]:
# Add this after your existing imports
import numpy as np
from arch import arch_model

import warnings
from arch.univariate.base import ConvergenceWarning
import warnings
from arch.univariate.base import ConvergenceWarning
import pandas as pd

def estimate_crisis_intensity(returns_series, window_size=252):
    """
    Estimate crisis intensity parameter using EGARCH conditional volatility with fallback
    """
    # Suppress convergence warnings
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=ConvergenceWarning)

        try:
            # Try EGARCH first
            model = arch_model(returns_series, vol='EGARCH', p=1, o=1, q=1, dist='t', rescale=False)
            results = model.fit(disp='off', options={'maxiter': 500})

            # Check if convergence was successful
            if results.convergence_flag != 0:
                raise ValueError("EGARCH did not converge properly")

            conditional_volatility = results.conditional_volatility

            # Convert to numpy array if it's a pandas Series
            if hasattr(conditional_volatility, 'values'):
                conditional_volatility = conditional_volatility.values

            # Handle NaN values
            conditional_volatility = np.nan_to_num(conditional_volatility, nan=0.0)

            # Normalize to [0,1] range
            sigma_min = np.min(conditional_volatility)
            sigma_max = np.max(conditional_volatility)

            if sigma_max - sigma_min == 0:
                crisis_intensity = np.zeros_like(conditional_volatility)
            else:
                crisis_intensity = (conditional_volatility - sigma_min) / (sigma_max - sigma_min)

            return crisis_intensity, sigma_min, sigma_max

        except:
            # Fallback to simple GARCH if EGARCH fails
            try:
                model = arch_model(returns_series, vol='GARCH', p=1, q=1, rescale=False)
                results = model.fit(disp='off')
                conditional_volatility = results.conditional_volatility

                if hasattr(conditional_volatility, 'values'):
                    conditional_volatility = conditional_volatility.values

                conditional_volatility = np.nan_to_num(conditional_volatility, nan=0.0)

                sigma_min = np.min(conditional_volatility)
                sigma_max = np.max(conditional_volatility)

                if sigma_max - sigma_min == 0:
                    crisis_intensity = np.zeros_like(conditional_volatility)
                else:
                    crisis_intensity = (conditional_volatility - sigma_min) / (sigma_max - sigma_min)

                return crisis_intensity, sigma_min, sigma_max

            except:
                # Final fallback: rolling volatility
                return estimate_crisis_intensity_simple(returns_series, window_size)

def estimate_crisis_intensity_simple(returns_series, window_size=252):
    """
    Simple crisis intensity using rolling volatility (fallback method)
    """
    # Calculate rolling volatility
    returns_df = pd.Series(returns_series)
    rolling_vol = returns_df.rolling(window=20, min_periods=5).std()
    rolling_vol = rolling_vol.fillna(rolling_vol.mean())

    # Normalize to [0,1]
    sigma_min = rolling_vol.min()
    sigma_max = rolling_vol.max()

    if sigma_max - sigma_min == 0:
        crisis_intensity = np.zeros_like(rolling_vol)
    else:
        crisis_intensity = (rolling_vol - sigma_min) / (sigma_max - sigma_min)

    return crisis_intensity.values, sigma_min, sigma_max



def preprocess_with_crisis_intensity(data, window=252):
    """
    Preprocess data and estimate crisis intensity for each sequence
    """
    crisis_intensities = []
    sigma_mins = []
    sigma_maxs = []

    for seq_idx in range(len(data)):
        # Get returns for this sequence (flatten across features)
        seq_returns = data[seq_idx].flatten()

        # Estimate crisis intensity
        c_values, sigma_min, sigma_max = estimate_crisis_intensity(seq_returns)

        # Take mean crisis intensity for the sequence
        mean_crisis = np.mean(c_values) if len(c_values) > 0 else 0.0
        crisis_intensities.append(mean_crisis)
        sigma_mins.append(sigma_min)
        sigma_maxs.append(sigma_max)

    return np.array(crisis_intensities), np.array(sigma_mins), np.array(sigma_maxs)

In [ ]:
class ConditionalGenerator(tf.keras.Model):
    def __init__(self, hidden_dim, z_dim, num_layers=3):
        super(ConditionalGenerator, self).__init__()
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim
        self.num_layers = num_layers

        # Crisis intensity embedding layer
        self.crisis_embedding = tf.keras.layers.Dense(hidden_dim//4, activation='relu')

        # Volatility scaler
        self.volatility_scaler = VolatilityScaler(z_dim)

        # LSTM stack
        self.lstm_layers = []
        for i in range(num_layers):
            self.lstm_layers.append(
                tf.keras.layers.LSTM(
                    hidden_dim,
                    return_sequences=True,
                    activation='tanh',
                    recurrent_activation='sigmoid'
                )
            )

        # Output projection
        self.output_layer = tf.keras.layers.Dense(hidden_dim, activation='linear')

    def call(self, noise_input, crisis_intensity, training=None):
        batch_size = tf.shape(noise_input)[0]
        seq_length = tf.shape(noise_input)[1]

        # Apply volatility scaling to noise
        scaled_noise = self.volatility_scaler(noise_input)

        # Embed crisis intensity
        c_embedding = self.crisis_embedding(crisis_intensity)  # [batch, emb_dim]

        # Repeat crisis embedding for each time step
        c_repeated = tf.tile(
            tf.expand_dims(c_embedding, 1),
            [1, seq_length, 1]
        )  # [batch, seq_length, emb_dim]

        # Concatenate noise with crisis conditioning
        combined_input = tf.concat([scaled_noise, c_repeated], axis=-1)

        # Pass through LSTM stack
        hidden_states = combined_input
        for lstm_layer in self.lstm_layers:
            hidden_states = lstm_layer(hidden_states, training=training)

        # Generate output
        output = self.output_layer(hidden_states)
        return output

In [ ]:
class ConditionalDiscriminator(tf.keras.Model):
    def __init__(self, hidden_dim, num_layers=3):
        super(ConditionalDiscriminator, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # LSTM backbone with spectral normalization
        self.lstm_layers = []
        for i in range(num_layers):
            lstm = tf.keras.layers.LSTM(
                hidden_dim,
                return_sequences=True,
                activation='tanh',
                recurrent_activation='sigmoid',
                unroll=True
            )
            # Apply spectral normalization to LSTM weights
            lstm = SNWrapper(lstm, power_iter=1)
            self.lstm_layers.append(lstm)

        # Self-attention pooling
        self.attention_pooling = SelfAttentionPooling(hidden_dim)

        # Dual heads for ACGAN approach
        self.adversarial_head = SNWrapper(
            tf.keras.layers.Dense(1, use_bias=False),
            power_iter=1
        )  # Real/fake classification
        self.crisis_regression_head = tf.keras.layers.Dense(
            1, activation='sigmoid'
        )  # Crisis intensity regression

    def call(self, latent_input, training=None):
        # Pass through LSTM stack
        hidden_states = latent_input
        for lstm_layer in self.lstm_layers:
            hidden_states = lstm_layer(hidden_states, training=training)

        # Apply self-attention pooling
        pooled_rep = self.attention_pooling(hidden_states)

        # Dual outputs
        adversarial_output = self.adversarial_head(pooled_rep)
        crisis_output = self.crisis_regression_head(pooled_rep)

        return adversarial_output, crisis_output

In [ ]:
class TimeGAN:
    def __init__(self, rolling_stats, **kwargs):
        super().__init__(**kwargs)
        self.rolling_stats = rolling_stats
        self.hidden_dim = parameters['hidden_dim']
        self.num_layers = parameters['num_layer']
        self.iterations = parameters['iterations']
        self.batch_size = parameters['batch_size']
        self.module_name = parameters['module']
        self.z_dim = parameters['z_dim']
        self.gamma = 1  # Adversarial weight
        self.dim = 2  # Dimensionality of input data (e.g., two stocks)

        # Build all sub-models first
        self.embedder_model = self.build_embedder()
        self.recovery_model = self.build_recovery()
        self.supervisor_model = self.build_supervisor()
        # Store crisis intensities
        self.crisis_intensities = crisis_intensities
        # Replace generator and discriminator with conditional versions
        self.generator_model = self.build_conditional_generator()
        self.discriminator_model = self.build_conditional_discriminator()
        # Define optimizers
        self.optimizer_E = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.optimizer_R = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.optimizer_S = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.optimizer_G = tf.keras.optimizers.Adam(learning_rate=1e-4,
                                            beta_1=0.5, beta_2=0.9)
        self.optimizer_D = tf.keras.optimizers.RMSprop(learning_rate=3e-5)

        # self.optimizer_D = tf.keras.optimizers.Adam(learning_rate=1e-4,
        #                                     beta_1=0.5, beta_2=0.9)

        # self.optimizer_G = tf.keras.optimizers.Adam(learning_rate=0.001)
        # self.optimizer_D = tf.keras.optimizers.Adam(learning_rate=0.0005)

        self.mse_loss = tf.keras.losses.MeanSquaredError()

        # Dummy initialization (after models are defined)
        print("Initializing model weights...")
        dummy_X = tf.random.normal([1, 60, self.dim])
        dummy_Z = tf.random.normal([1, 60, self.z_dim])
        dummy_H = tf.random.normal([1, 60, self.hidden_dim])
        _ = self.embedder_model(dummy_X)
        _ = self.recovery_model(dummy_H)
        _ = self.generator_model(dummy_Z)
        _ = self.supervisor_model(dummy_H)
        _ = self.discriminator_model(dummy_H)
        print("Model weights initialized.")

    def build_embedder(self):
        inputs = Input(shape=(None, self.dim))
        x = inputs
        for _ in range(self.num_layers):
            x = LSTM(self.hidden_dim, return_sequences=True)(x)
            #  x = layers.GRU(self.hidden_dim, return_sequences=True)(x)
        # outputs = Dense(self.hidden_dim, activation='sigmoid')(x) #was in the best attempt
        outputs = Dense(self.hidden_dim, activation='linear')(x)
        # outputs = Dense(self.hidden_dim, activation='tanh')(x)
        return tf.keras.Model(inputs, outputs, name='Embedder')

    def build_recovery(self):
        inputs = tf.keras.Input((None, self.hidden_dim))
        x = inputs
        for _ in range(self.num_layers):
            x = tf.keras.layers.LSTM(self.hidden_dim,
                                  return_sequences=True,
                                  activation='swish')(x)  # Better gradient flow
        # Direct output without artificial scaling
        outputs = tf.keras.layers.Dense(self.dim, activation='linear')(x)
        return tf.keras.Model(inputs, outputs, name="Recovery")

    def build_supervisor(self):
        inputs = Input(shape=(None, self.hidden_dim))
        x = inputs
        for _ in range(self.num_layers - 1):
            x = LSTM(self.hidden_dim, return_sequences=True)(x)
            #  x = layers.GRU(self.hidden_dim, return_sequences=True)(x)
        # outputs = Dense(self.hidden_dim, activation='sigmoid')(x)
        # outputs = Dense(self.hidden_dim, activation='tanh')(x)
        outputs = Dense(self.hidden_dim, activation='linear')(x)
        return tf.keras.Model(inputs, outputs, name='Supervisor')

    def build_conditional_generator(self):
        return ConditionalGenerator(
            hidden_dim=self.hidden_dim,
            z_dim=self.z_dim,
            num_layers=self.num_layers
        )

    def build_conditional_discriminator(self):
        return ConditionalDiscriminator(
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers
        )

In [ ]:
# Extract time information at the beginning
ori_time, max_seq_len = extract_time(ori_data)

In [ ]:
def rolling_window_standardize(data, window=252):
    """
    Faster rolling standardization using pandas.

    Args:
        data: (num_sequences, seq_len, num_features) numpy array
        window: rolling window size

    Returns:
        standardized_data: same shape as input
        rolling_stats: statistics for descaling
    """
    num_sequences, seq_len, num_features = data.shape
    standardized_data = np.zeros_like(data)
    rolling_means = np.zeros_like(data)
    rolling_stds = np.zeros_like(data)

    for feat_idx in range(num_features):
        for seq_idx in range(num_sequences):
            # Convert to pandas Series for efficient rolling operations
            series = pd.Series(data[seq_idx, :, feat_idx])

            # Compute rolling statistics with minimum periods
            rolling_mean = series.rolling(window=window, min_periods=10).mean()
            rolling_std = series.rolling(window=window, min_periods=10).std()

            # Fill initial NaN values with expanding window
            rolling_mean = rolling_mean.fillna(series.expanding().mean())
            rolling_std = rolling_std.fillna(series.expanding().std())
            rolling_std = rolling_std.fillna(1e-8).clip(lower=1e-8)

            # Store statistics
            rolling_means[seq_idx, :, feat_idx] = rolling_mean.values
            rolling_stds[seq_idx, :, feat_idx] = rolling_std.values

            # Standardize
            standardized_data[seq_idx, :, feat_idx] = (
                (series - rolling_mean) / rolling_std
            ).values

    return standardized_data, {
        'means': rolling_means,
        'stds': rolling_stds
    }

In [ ]:
def preprocess_data_rolling(data, window=252, train_ratio=0.8):
    """
    Split data and apply rolling window standardization separately.

    Args:
        data: (num_sequences, seq_len, num_features) numpy array
        window: rolling window size for standardization
        train_ratio: fraction of data for training

    Returns:
        train_data: standardized training data
        test_data: standardized test data
        train_stats: rolling statistics for training data
        test_stats: rolling statistics for test data
    """
    # Split first, then standardize separately
    n_sequences = len(data)
    split_idx = int(n_sequences * train_ratio)

    train_raw = data[:split_idx]
    test_raw = data[split_idx:]

    # Apply rolling standardization separately
    train_data, train_stats = rolling_window_standardize(train_raw, window)
    test_data, test_stats = rolling_window_standardize(test_raw, window)

    return train_data, test_data, train_stats, test_stats

# Usage in your main pipeline
train_data, test_data, train_stats, test_stats = preprocess_data_rolling(
    ori_data, window=252, train_ratio=0.8
)

# Update your global variables
ori_data = np.concatenate([train_data, test_data], axis=0)
ori_time, max_seq_len = extract_time(ori_data)

In [ ]:
# ------------------------------------------------------------------
# rolling-stat tensors that every batch-generator call will re-use
# ------------------------------------------------------------------
roll_means = np.concatenate([train_stats['means'],  test_stats['means']], 0)
roll_stds  = np.concatenate([train_stats['stds'],   test_stats['stds']],  0)

rolling_stats = {
    "means": roll_means,   # shape (N, L, F)
    "stds" : roll_stds     # same shape
}

In [ ]:
# Estimate crisis intensities for your data
print("Estimating crisis intensities...")
crisis_intensities = []
for seq_idx in range(len(ori_data)):
    # Get returns for this sequence (flatten across features)
    seq_returns = ori_data[seq_idx].flatten()

    # Estimate crisis intensity using EGARCH
    c_values, sigma_min, sigma_max = estimate_crisis_intensity(seq_returns)

    # Take mean crisis intensity for the sequence
    mean_crisis = np.mean(c_values) if len(c_values) > 0 else 0.0
    crisis_intensities.append(mean_crisis)

crisis_intensities = np.array(crisis_intensities)
print(f"Crisis intensities range: {crisis_intensities.min():.3f} to {crisis_intensities.max():.3f}")


In [ ]:
model = TimeGAN(rolling_stats)

In [ ]:
def descale_rolling(x_scaled, rolling_stats, is_training=True):
    """
    Descale using rolling window statistics.

    Args:
        x_scaled: (batch_size, seq_len, num_features) scaled data
        rolling_stats: dict with 'means' and 'stds' arrays
        is_training: whether to use training or test statistics

    Returns:
        descaled data in original scale
    """
    stats_key = 'train' if is_training else 'test'
    means = rolling_stats[stats_key]['means']
    stds = rolling_stats[stats_key]['stds']

    # Convert to TensorFlow tensors if needed
    if isinstance(x_scaled, tf.Tensor):
        means_tf = tf.constant(means, dtype=x_scaled.dtype)
        stds_tf = tf.constant(stds, dtype=x_scaled.dtype)

        # For batch processing, tile statistics to match batch size
        batch_size = tf.shape(x_scaled)[0]
        means_tiled = tf.tile(tf.expand_dims(means_tf, 0), [batch_size, 1, 1])
        stds_tiled = tf.tile(tf.expand_dims(stds_tf, 0), [batch_size, 1, 1])

        return x_scaled * stds_tiled + means_tiled
    else:
        # NumPy version
        return x_scaled * stds + means

# Store rolling statistics globally for use in model
global_rolling_stats = {
    'train': train_stats,
    'test': test_stats
}

In [ ]:
@tf.function
def train_embedder(X_mb, T_mb):
    with tf.GradientTape() as tape:
        H = model.embedder_model(X_mb)  # Encode original data
        X_tilde = model.recovery_model(H)  # Decode back to original space
        E_loss_T0 = tf.reduce_mean(model.mse_loss(X_mb, X_tilde))  # Reconstruction Loss
        # E_loss_T0 = masked_mse(X_mb, X_tilde, T_mb)
        E_loss = 10 * tf.sqrt(E_loss_T0)  # Maintain loss scaling

    gradients = tape.gradient(E_loss,
                              model.embedder_model.trainable_variables +
                              model.recovery_model.trainable_variables)

    gradients, _ = tf.clip_by_global_norm(gradients, 0.5)  # Prevent exploding gradients

    # Apply gradient updates for both the embedder and recovery networks
    model.optimizer_E.apply_gradients(zip(gradients,
                                          model.embedder_model.trainable_variables +
                                          model.recovery_model.trainable_variables))

    return E_loss

In [ ]:
@tf.function
def train_embedder_step_joint(X_mb: tf.Tensor, T_mb: tf.Tensor) -> tf.Tensor:
    """
    Joint training embedder step (optional).
    We continue fine-tuning the embedder + recovery networks
    to improve reconstruction.
    """
    with tf.GradientTape() as tape:
        # Encode real data into H
        H = model.embedder_model(X_mb)
        # Decode back to original space
        X_tilde = model.recovery_model(H)
        # Reconstruction Loss
        E_loss_T0 = model.mse_loss(X_mb, X_tilde)
        # E_loss_T0 = masked_mse(X_mb, X_tilde, T_mb)
        # Scale up the embedder loss slightly
        E_loss = 10 * tf.sqrt(E_loss_T0 + 1e-9)

    # Compute gradients
    e_vars = model.embedder_model.trainable_variables + model.recovery_model.trainable_variables
    e_grads = tape.gradient(E_loss, e_vars)
    e_grads, _ = tf.clip_by_global_norm(e_grads, 0.5)

    # Apply gradients
    model.optimizer_E.apply_gradients(zip(e_grads, e_vars))

    return E_loss

In [ ]:
@tf.function
def train_supervised_step(X_mb, T_mb):
    """Train the supervisor network to predict next latent state."""
    with tf.GradientTape() as tape:
        H = model.embedder_model(X_mb)
        H_hat_supervise = model.supervisor_model(H)
        G_loss_S = tf.reduce_mean(tf.keras.losses.MeanSquaredError()(H[:, :-1, :], H_hat_supervise[:, 1:, :]))

    gradients = tape.gradient(G_loss_S, model.supervisor_model.trainable_variables)
    model.optimizer_S.apply_gradients(zip(gradients, model.supervisor_model.trainable_variables))
    return G_loss_S

In [ ]:
# ------------------------------------------------------------
# SAFE batched_acf  – works inside @tf.function
# ------------------------------------------------------------
@tf.function
def batched_acf(x, max_lag: int):
    """
    x : (B,T,F) float32/64
    returns tensor shape (max_lag, F) – ACF averaged over batch
    """
    B = tf.shape(x)[0]
    T = tf.shape(x)[1]
    F = tf.shape(x)[2]

    x_mean = tf.reduce_mean(x, axis=1, keepdims=True)
    xc     = x - x_mean
    var    = tf.reduce_mean(tf.square(xc), axis=1, keepdims=True)   # (B,1,F)
    var    = tf.maximum(var, 1e-8)

    # TensorArray to collect results inside the loop
    ta = tf.TensorArray(dtype=x.dtype, size=max_lag, dynamic_size=False,
                        clear_after_read=False)

    T_f = tf.cast(T, x.dtype)

    def body(k, ta_):
        num = tf.reduce_sum(xc[:, :-k, :] * xc[:, k:, :], axis=1)           # (B,F)
        denom = (T_f - tf.cast(k, x.dtype)) * var[:, 0, :]                  # (B,F)
        ta_ = ta_.write(k-1, tf.reduce_mean(num / denom, axis=0))           # (F,)
        return k+1, ta_

    _, ta_final = tf.while_loop(
        lambda k, *_: k <= max_lag,
        body,
        loop_vars=(1, ta)
    )

    # stack -> (max_lag, F)
    return ta_final.stack()

In [ ]:
def tail_loss_two_sided(r_real, r_fake, q=0.05,
                        w_left=1., w_right=.5):
    rr = tf.reshape(r_real, [-1, tf.shape(r_real)[-1]])
    rf = tf.reshape(r_fake, [-1, tf.shape(r_fake)[-1]])

    # ---- VaR/ES of the REAL distribution only -----------------
    VaR_l = tfp.stats.percentile(rr, 100*q,      axis=0)
    VaR_r = tfp.stats.percentile(rr, 100*(1-q),  axis=0)

    ES_r_l = tf.reduce_mean(tf.boolean_mask(rr, rr <= VaR_l), axis=0)
    ES_f_l = tf.reduce_mean(tf.boolean_mask(rf, rf <= VaR_l), axis=0)  # <-- SAME threshold

    ES_r_r = tf.reduce_mean(tf.boolean_mask(rr, rr >= VaR_r), axis=0)
    ES_f_r = tf.reduce_mean(tf.boolean_mask(rf, rf >= VaR_r), axis=0)  # <-- SAME threshold

    sig = tf.math.reduce_std(rr, axis=0) + 1e-8          # real σ

    loss_left  = tf.square((ES_r_l - ES_f_l) / sig)
    loss_right = tf.square((ES_r_r - ES_f_r) / sig)

    return tf.reduce_mean(w_left*loss_left + w_right*loss_right)

In [ ]:
def rolling_std(x: tf.Tensor, win: int, mask: tf.Tensor = None):
    """Causal rolling σ over last *win* steps.
    If *mask* is supplied (shape == x), treats 0s as missing.
    Args
    -----
    x    : (B,T,F) float32/64  – returns in real space
    win  : int                  – window length
    mask : (B,T,1|F) or None   – 0/1 mask
    """
    if mask is None:
        mask = tf.ones_like(x)
    w   = tf.cast(mask, x.dtype)
    num = tf.nn.avg_pool1d(x * w, win, 1, "SAME", data_format="NWC")
    den = tf.nn.avg_pool1d(w,      win, 1, "SAME", data_format="NWC") + 1e-8
    mean = num / den
    var  = tf.nn.avg_pool1d(w * tf.square(x - mean), win, 1, "SAME", data_format="NWC") / den
    return tf.sqrt(var + 1e-7)

In [ ]:
# ------------------------------------------------------------
# heteroskedasticity loss
# ------------------------------------------------------------
def heteroskedasticity_loss(real_r: tf.Tensor, fake_r: tf.Tensor, window: int = 20):
    """L1 difference of rolling σ  +  ACF of squared returns."""
    real_vol = rolling_std(real_r, window)
    fake_vol = rolling_std(fake_r, window)
    vol_loss = tf.reduce_mean(tf.abs(real_vol - fake_vol))

    real_sq, fake_sq = tf.square(real_r), tf.square(fake_r)
    real_acf = batched_acf(real_sq, 10)
    fake_acf = batched_acf(fake_sq, 10)
    cluster_loss = tf.reduce_mean(tf.square(real_acf - fake_acf))
    return vol_loss + cluster_loss

In [ ]:
def descale_time_varying(x_scaled, mu, sigma):
    # x_scaled , mu , sigma  are all (B,T,F)
    return x_scaled * sigma + mu

In [ ]:
@tf.function
def train_conditional_generator_step(
    X_mb: tf.Tensor,          # (B,T,F)  scaled real log‑returns
    T_mb: tf.Tensor,          # (B,)     true lengths
    Z_mb: tf.Tensor,          # (B,T,z)  latent noise
    crisis_mb: tf.Tensor,     # (B,1)    crisis intensity targets
    mu_mb,                    # (B,T,F) rolling means
    sig_mb,                   # (B,T,F) rolling stds
    *,
    step: tf.Tensor,          # global step (int32/int64)
    roll_window: int = 25,
):
    # ————————————————— weights (tune as you like) —————————————————
    w_S, w_V      = 10.0, 10.0
    w_mu, w_tail  = 15.0, 15.0
    w_sigma, w_vol= 10.0, 20.0
    w_acf, w_het  = 15.0, 20.0
    w_C           = 1.0          # NEW: crisis intensity matching weight
    w_crisis      = 0.5          # NEW: crisis-specific losses weight

    mask = tf.expand_dims(tf.sequence_mask(T_mb, tf.shape(X_mb)[1], dtype=X_mb.dtype), -1)

    with tf.GradientTape() as tape:
        # ---- forward ----
        H_real = model.embedder_model(X_mb, training=False)
        E_hat  = model.generator_model(Z_mb, crisis_mb, training=True)  # NOW CONDITIONAL
        H_hat  = model.supervisor_model(E_hat, training=True)
        H_sup  = model.supervisor_model(H_real, training=False)

        # DUAL-HEAD discriminator outputs
        Y_fake, crisis_pred_fake = model.discriminator_model(H_hat, training=False)

        r_fake_scaled = model.recovery_model(H_hat, training=False) * mask

        # --------- inverse-scale with time-varying stats ------------
        r_real = descale_time_varying(X_mb*mask,  mu_mb,  sig_mb)
        r_fake = descale_time_varying(r_fake_scaled, mu_mb, sig_mb)

        # ===== ORIGINAL TimeGAN losses =====
        G_loss_U = -tf.reduce_mean(Y_fake)
        G_loss_S = tf.sqrt(tf.reduce_mean(tf.keras.losses.mse(H_real[:,1:,:], H_sup[:,:-1,:])))
        G_loss_V = (tf.reduce_mean(tf.abs(tf.math.reduce_std(H_hat,0) - tf.math.reduce_std(H_real,0))) +
                     tf.reduce_mean(tf.abs(tf.reduce_mean(H_hat,0) - tf.reduce_mean(H_real,0))))

        # ===== ORIGINAL financial losses =====
        # mean‑matching, VaR/ES, global σ
        mu_r = tf.reduce_mean(r_real, axis=[0,1]); mu_f = tf.reduce_mean(r_fake, axis=[0,1])
        L_mu = tf.reduce_mean(tf.square(mu_r - mu_f))
        Tail_main = tail_loss_two_sided(r_real, r_fake, q=0.05, w_left=1.5, w_right=1.0)
        sigma_r = tf.math.reduce_std(r_real); sigma_f = tf.math.reduce_std(r_fake)
        margin = 1.1
        L_sigma = tf.where(sigma_f > sigma_r*margin,
                           (sigma_f-sigma_r)/sigma_r,
                           (sigma_r*margin-sigma_f)/(sigma_r*0.5))
        L_sigma = tf.reduce_mean(L_sigma)

        # rolling σ + ACF
        L_vol = tf.reduce_mean(tf.abs(rolling_std(r_real,roll_window,mask) -
                                       rolling_std(r_fake,roll_window,mask)))

        acf_real = batched_acf(r_real, 20); acf_fake = batched_acf(r_fake, 20)
        acf_weights = tf.concat([[2.], tf.ones([19])],0)
        L_acf = tf.reduce_mean(tf.abs(acf_real - acf_fake) * acf_weights[:,None,None])

        # heteroskedasticity term
        L_het = heteroskedasticity_loss(r_real, r_fake, window=20)

        # ===== NEW C-G-GAN losses =====
        # Crisis intensity matching loss
        G_loss_C = tf.reduce_mean(tf.square(crisis_pred_fake - crisis_mb))

        # Crisis-specific losses (enhanced tail risk, volatility clustering, correlations)
        crisis_specific_loss = compute_crisis_specific_loss(r_real, r_fake, crisis_mb)

        # scale-free diagnostics
        dmu_over_sig = tf.reduce_mean(tf.abs(mu_r - mu_f) /
                                      (sigma_r + 1e-8))
        sig_ratio    = tf.reduce_mean(sigma_f / (sigma_r + 1e-8))

        # ===== TOTAL LOSS (with C-G-GAN components) =====
        G_total = (G_loss_U + w_S*G_loss_S + w_V*G_loss_V + w_mu*L_mu + w_tail*Tail_main +
                   w_sigma*L_sigma + w_vol*L_vol + w_acf*L_acf + w_het*L_het +
                   w_C*G_loss_C + w_crisis*crisis_specific_loss)  # NEW TERMS

    vars_G = model.generator_model.trainable_variables + model.supervisor_model.trainable_variables
    grads  = tape.gradient(G_total, vars_G)

    model.optimizer_G.apply_gradients(zip(grads, vars_G))

    return (G_total,
            G_loss_U,  G_loss_S,  G_loss_V,
            L_mu, Tail_main, L_sigma,
            L_vol, L_acf, L_het,
            G_loss_C, crisis_specific_loss,  # NEW RETURNS
            dmu_over_sig, sig_ratio
    )

# ===== HELPER FUNCTION: Crisis-Specific Loss =====
@tf.function
def compute_crisis_specific_loss(r_real, r_fake, crisis_mb):
    """
    Compute crisis-specific losses that focus on extreme events

    Args:
        r_real: (B,T,F) real returns
        r_fake: (B,T,F) synthetic returns
        crisis_mb: (B,1) crisis intensity levels
    """
    # Weight samples by crisis intensity (higher weight for crisis periods)
    crisis_weights = tf.expand_dims(crisis_mb, -1)  # (B,1,1)

    # Enhanced tail risk matching (weighted by crisis intensity)
    var_01 = tfp.stats.percentile(r_real, 1.0, axis=[0,1])   # 1% VaR
    var_99 = tfp.stats.percentile(r_real, 99.0, axis=[0,1])  # 99% VaR

    # Expected shortfall for extreme tails
    es_real_01 = tf.reduce_mean(tf.boolean_mask(r_real, r_real <= var_01))
    es_fake_01 = tf.reduce_mean(tf.boolean_mask(r_fake, r_fake <= var_01))
    es_real_99 = tf.reduce_mean(tf.boolean_mask(r_real, r_real >= var_99))
    es_fake_99 = tf.reduce_mean(tf.boolean_mask(r_fake, r_fake >= var_99))

    tail_loss = (tf.square(es_real_01 - es_fake_01) +
                 tf.square(es_real_99 - es_fake_99))

    # Volatility clustering (crisis periods should have higher persistence)
    vol_real = tf.abs(r_real)
    vol_fake = tf.abs(r_fake)

    # Autocorrelation of absolute returns (volatility clustering indicator)
    vol_acf_real = batched_acf(vol_real, 10)
    vol_acf_fake = batched_acf(vol_fake, 10)
    vol_clustering_loss = tf.reduce_mean(tf.square(vol_acf_real - vol_acf_fake))

    # Cross-asset correlation during crisis (should increase)
    if tf.shape(r_real)[-1] > 1:  # Multi-asset case
        # Compute rolling correlations
        corr_real = compute_rolling_correlation(r_real, window=20)
        corr_fake = compute_rolling_correlation(r_fake, window=20)
        correlation_loss = tf.reduce_mean(tf.square(corr_real - corr_fake))
    else:
        correlation_loss = 0.0

    # Maximum drawdown matching (important for crisis modeling)
    dd_real = compute_max_drawdown(r_real)
    dd_fake = compute_max_drawdown(r_fake)
    drawdown_loss = tf.square(dd_real - dd_fake)

    # Weight all losses by crisis intensity
    total_crisis_loss = crisis_weights * (
        2.0 * tail_loss +           # Heavy weight on tail events
        1.5 * vol_clustering_loss + # Volatility clustering
        1.0 * correlation_loss +    # Cross-asset correlations
        1.0 * drawdown_loss         # Maximum drawdowns
    )

    return tf.reduce_mean(total_crisis_loss)

@tf.function
def compute_rolling_correlation(returns, window=20):
    """Compute rolling correlation between assets"""
    if tf.shape(returns)[-1] < 2:
        return tf.constant(0.0)

    # Simple correlation between first two assets
    r1 = returns[:, :, 0]  # (B,T)
    r2 = returns[:, :, 1]  # (B,T)

    # Rolling correlation using conv1d
    r1_mean = tf.nn.avg_pool1d(tf.expand_dims(r1, -1), window, 1, "SAME")[:,:,0]
    r2_mean = tf.nn.avg_pool1d(tf.expand_dims(r2, -1), window, 1, "SAME")[:,:,0]

    r1_centered = r1 - r1_mean
    r2_centered = r2 - r2_mean

    cov = tf.nn.avg_pool1d(tf.expand_dims(r1_centered * r2_centered, -1), window, 1, "SAME")[:,:,0]
    var1 = tf.nn.avg_pool1d(tf.expand_dims(r1_centered**2, -1), window, 1, "SAME")[:,:,0]
    var2 = tf.nn.avg_pool1d(tf.expand_dims(r2_centered**2, -1), window, 1, "SAME")[:,:,0]

    corr = cov / (tf.sqrt(var1 * var2) + 1e-8)
    return tf.reduce_mean(corr)

@tf.function
def compute_max_drawdown(returns):
    """Compute maximum drawdown from returns"""
    # Convert returns to cumulative wealth
    cum_returns = tf.cumsum(returns, axis=1)

    # Running maximum
    running_max = tf.scan(
        fn=lambda acc, x: tf.maximum(acc, x),
        elems=tf.transpose(cum_returns, [1, 0, 2]),
        initializer=cum_returns[:, 0, :]
    )
    running_max = tf.transpose(running_max, [1, 0, 2])

    # Drawdown
    drawdown = cum_returns - running_max
    max_drawdown = tf.reduce_min(drawdown, axis=1)  # Most negative value

    return tf.reduce_mean(max_drawdown)

# ===== HELPER FUNCTION: Time-varying descaling =====
@tf.function
def descale_time_varying(x_scaled, mu_mb, sig_mb):
    """Descale using time-varying rolling statistics"""
    return x_scaled * sig_mb + mu_mb


In [ ]:
@tf.function
def train_conditional_discriminator_step(X_mb, Z_mb, crisis_mb):
    """
    One conditional critic update under the WGAN-SN setting with crisis regression.
    The discriminator learns both adversarial classification and crisis intensity prediction.

    Args:
        X_mb: (B,T,F) real data batch
        Z_mb: (B,T,z) noise batch
        crisis_mb: (B,1) crisis intensity targets
    """

    with tf.GradientTape() as disc_tape:
        # ---------- real latent path ---------------------------------
        H_real = tf.stop_gradient(model.embedder_model(X_mb))              # embedder is *frozen*
        Y_real, crisis_pred_real = model.discriminator_model(H_real)       # DUAL outputs

        # ---------- fake latent path  (GRADIENTS STOPPED here) -------
        E_hat  = tf.stop_gradient(model.generator_model(Z_mb, crisis_mb))  # NOW CONDITIONAL
        H_fake = tf.stop_gradient(model.supervisor_model(E_hat))
        Y_fake, crisis_pred_fake = model.discriminator_model(H_fake)       # DUAL outputs

        # ---------- Wasserstein adversarial loss --------------------
        D_loss_adv = tf.reduce_mean(Y_fake) - tf.reduce_mean(Y_real)

        # ---------- Crisis regression loss ---------------------------
        # Real data: predict crisis intensity from real latent representations
        D_loss_crisis_real = tf.reduce_mean(tf.square(crisis_pred_real - crisis_mb))

        # Fake data: predict crisis intensity from synthetic latent representations
        D_loss_crisis_fake = tf.reduce_mean(tf.square(crisis_pred_fake - crisis_mb))

        # Combined crisis regression loss
        D_loss_crisis = D_loss_crisis_real + D_loss_crisis_fake

        # ---------- Total discriminator loss -------------------------
        lambda_crisis = 1.0  # Weight for crisis regression loss
        D_loss_total = D_loss_adv + lambda_crisis * D_loss_crisis

    # Compute gradients for discriminator only
    grads = disc_tape.gradient(
        D_loss_total, model.discriminator_model.trainable_variables
    )
    grads, _ = tf.clip_by_global_norm(grads, 1.5)     # ‖g‖₂ ≤ 1.5

    # Apply gradients to discriminator
    model.optimizer_D.apply_gradients(
        zip(grads, model.discriminator_model.trainable_variables)
    )

    return D_loss_total, D_loss_adv, D_loss_crisis

In [ ]:
train_time, _ = extract_time(train_data)
test_time , _ = extract_time(test_data)
_, max_seq_len = extract_time(train_data)        # or use a constant

In [ ]:
@tf.function
def critic_grad_norm(x_real):
    with tf.GradientTape() as tape:
        tape.watch(x_real)  # Watch the input tensor
        h = model.embedder_model(x_real)  # latent representation
        y = model.discriminator_model(h)
    grads = tape.gradient(y, h)  # Gradient of critic output wrt latent h
    # Compute L2 norm over batch and time dimensions: shape (batch, time, features)
    grad_norms = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2]) + 1e-8)
    # Return average grad norm over batch
    return tf.reduce_mean(grad_norms)

In [ ]:
def generate_synthetic_samples(model,
                               n_samples: int,
                               ref_real: np.ndarray,
                               max_len: int):
    """
    Create `n_samples` synthetic sequences.

    Parameters
    ----------
    model      : your TimeGAN instance
    n_samples  : how many sequences you want back
    ref_real   : a *real* dataset of shape (N, L, F) –
                 only used for their individual lengths
    max_len    : integer, padding length you used in `random_generator`

    Returns
    -------
    np.ndarray of shape (n_samples, L_i, F) – the same
    per-sequence lengths as in `ref_real[:n_samples]`
    """
    # 1) grab the first n length values from whatever set you pass in
    seq_lens = [len(seq) for seq in ref_real[:n_samples]]

    synthetic_chunks = []
    for i in range(0, n_samples, model.batch_size):
        batch_lens = seq_lens[i : i + model.batch_size]

        # build noise that matches those lengths
        Z_mb = random_generator(len(batch_lens),
                                model.z_dim,
                                batch_lens,
                                max_len)

        # forward through G → S → R
        E_hat = model.generator_model(Z_mb, training=False)
        H_hat = model.supervisor_model(E_hat, training=False)
        X_hat = model.recovery_model(H_hat, training=False)

        synthetic_chunks.append(X_hat.numpy())

    return np.concatenate(synthetic_chunks, axis=0)[:n_samples]

In [ ]:
@tf.function
def critic_gap_no_grad(x_real, z_noise):
    """ E[D_real] − E[D_fake]  (no gradients). """
    h_r = tf.stop_gradient(model.embedder_model(x_real))
    h_f = tf.stop_gradient(model.supervisor_model(
                               model.generator_model(z_noise)))
    d_r = model.discriminator_model(h_r, training=False)
    d_f = model.discriminator_model(h_f, training=False)
    return tf.reduce_mean(d_r) - tf.reduce_mean(d_f)

In [ ]:
init_sig = tf.cast(tf.math.reduce_std(train_data, axis=[0, 1]), tf.float32)
vs_layer = VolatilityScaler(z_dim, init_sig=init_sig)    # ← pass here

In [ ]:
def train_conditional_timegan(model, ori_data, ori_time, rolling_stats, crisis_intensities):
    """
    Complete training pipeline for Conditional TimeGAN with crisis modeling
    (without discriminator and generator warm-up phases)

    Args:
        model: ConditionalTimeGAN instance
        ori_data: preprocessed sequence data
        ori_time: sequence lengths
        rolling_stats: rolling standardization statistics
        crisis_intensities: estimated crisis parameters for each sequence
    """
    print("Starting Conditional TimeGAN training...")

    # Phase 1: Embedder-Recovery Pre-training (2000 iterations)
    print("Phase 1: Training Embedder-Recovery...")
    for itt in range(20000):
        X_mb, T_mb, mu_mb, sigma_mb, crisis_mb = batch_generator(
            ori_data, ori_time,
            rolling_stats['means'], rolling_stats['stds'],
            crisis_intensities, model.batch_size
        )
        X_mb = tf.constant(X_mb, dtype=tf.float32)

        E_loss = train_embedder(model, X_mb, T_mb)

        if itt % 500 == 0:
            print(f"Iteration {itt}, Embedder Loss: {E_loss:.4f}")

    # Phase 2: Supervisor Pre-training (2000 iterations)
    print("Phase 2: Training Supervisor...")
    for itt in range(20000):
        X_mb, T_mb, mu_mb, sigma_mb, crisis_mb = batch_generator(
            ori_data, ori_time,
            rolling_stats['means'], rolling_stats['stds'],
            crisis_intensities, model.batch_size
        )
        X_mb = tf.constant(X_mb, dtype=tf.float32)

        S_loss = train_supervised_step(model, X_mb, T_mb)

        if itt % 500 == 0:
            print(f"Iteration {itt}, Supervisor Loss: {S_loss:.4f}")

    # Phase 3: Joint Adversarial Training (main training loop)
    print("Phase 3: Joint conditional adversarial training...")
    for itt in range(model.iterations):
        # Train discriminator multiple times (2x per generator update)
        d_losses = []
        for _ in range(2):
            X_mb, T_mb, mu_mb, sigma_mb, crisis_mb = batch_generator(
                ori_data, ori_time,
                rolling_stats['means'], rolling_stats['stds'],
                crisis_intensities, model.batch_size
            )
            Z_mb = random_generator(model.batch_size, model.z_dim, T_mb, max(T_mb))

            X_mb = tf.constant(X_mb, dtype=tf.float32)
            Z_mb = tf.constant(Z_mb, dtype=tf.float32)
            crisis_mb = tf.constant(crisis_mb, dtype=tf.float32)

            D_loss, D_loss_adv, D_loss_crisis = train_conditional_discriminator_step(
                X_mb, Z_mb, crisis_mb
            )
            d_losses.append(D_loss)

        # Train generator once
        X_mb, T_mb, mu_mb, sigma_mb, crisis_mb = batch_generator(
            ori_data, ori_time,
            rolling_stats['means'], rolling_stats['stds'],
            crisis_intensities, model.batch_size
        )
        Z_mb = random_generator(model.batch_size, model.z_dim, T_mb, max(T_mb))

        X_mb = tf.constant(X_mb, dtype=tf.float32)
        Z_mb = tf.constant(Z_mb, dtype=tf.float32)
        crisis_mb = tf.constant(crisis_mb, dtype=tf.float32)

        G_losses = train_conditional_generator_step(
            X_mb, Z_mb, T_mb, crisis_mb, mu_mb, sigma_mb, step=itt
        )

        # Unpack generator losses
        (G_total, G_loss_U, G_loss_S, G_loss_V, L_mu, Tail_main, L_sigma,
         L_vol, L_acf, L_het, G_loss_C, crisis_specific_loss,
         dmu_over_sig, sig_ratio) = G_losses

        # Periodic logging
        if itt % 1000 == 0:
            avg_d_loss = tf.reduce_mean(d_losses)
            print(f"\nIteration {itt}")
            print(f"  Discriminator - Total: {avg_d_loss:.4f}, Adv: {D_loss_adv:.4f}, Crisis: {D_loss_crisis:.4f}")
            print(f"  Generator - Total: {G_total:.4f}")
            print(f"    Adversarial: {G_loss_U:.4f}, Supervised: {G_loss_S:.4f}")
            print(f"    Crisis Matching: {G_loss_C:.4f}, Crisis Specific: {crisis_specific_loss:.4f}")
            print(f"    Financial - Mu: {L_mu:.4f}, Tail: {Tail_main:.4f}, Vol: {L_vol:.4f}")
            print(f"    Diagnostics - μ/σ: {dmu_over_sig:.4f}, σ_ratio: {sig_ratio:.4f}")

        # Optional: Model checkpointing
        if itt % 5000 == 0 and itt > 0:
            print(f"Saving checkpoint at iteration {itt}")
            # Save model weights here if needed

    print("Conditional training completed!")
    return model

In [ ]:
# For Conditional TimeGAN
class ConditionalTimeGAN(TimeGAN):
    def __init__(self, rolling_stats, crisis_intensities, **kwargs):
        super().__init__(rolling_stats, **kwargs)
        self.crisis_intensities = crisis_intensities

        # Replace with conditional versions
        self.generator_model = ConditionalGenerator(
            hidden_dim=self.hidden_dim,
            output_dim=self.hidden_dim,
            num_layers=self.num_layers
        )
        self.discriminator_model = ConditionalDiscriminator(
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers
        )

        # Re-initialize weights
        self._initialize_conditional_weights()

    def _initialize_conditional_weights(self):
        print("Initializing conditional model weights...")
        dummy_Z = tf.random.normal([1, 60, self.z_dim])
        dummy_H = tf.random.normal([1, 60, self.hidden_dim])
        dummy_c = tf.random.uniform([1, 1], 0.0, 1.0)

        _ = self.generator_model(dummy_Z, dummy_c)
        _ = self.discriminator_model(dummy_H)
        print("Conditional model weights initialized.")

# Define conditional model
model = ConditionalTimeGAN(rolling_stats, crisis_intensities)

In [ ]:
# Main execution
if __name__ == "__main__":
    # Estimate crisis intensities for your data
    print("Estimating crisis intensities...")
    crisis_intensities, sigma_mins, sigma_maxs = preprocess_with_crisis_intensity(ori_data)
    print(f"Crisis intensities range: {crisis_intensities.min():.3f} to {crisis_intensities.max():.3f}")

    # Initialize conditional model
    conditional_model = ConditionalTimeGAN(rolling_stats, crisis_intensities)

    # Train the model
    trained_conditional_model = train_conditional_timegan(
        conditional_model, ori_data, ori_time, rolling_stats, crisis_intensities
    )

    print("Training completed successfully!")

In [ ]:
# Estimate crisis intensities for your data
print("Estimating crisis intensities...")
crisis_intensities, sigma_mins, sigma_maxs = preprocess_with_crisis_intensity(ori_data)
print(f"Crisis intensities range: {crisis_intensities.min():.3f} to {crisis_intensities.max():.3f}")

# Initialize and train conditional model
conditional_model = ConditionalTimeGAN(rolling_stats, crisis_intensities)
trained_conditional_model = train_conditional_timegan(
    conditional_model, ori_data, ori_time, rolling_stats, crisis_intensities
)

def generate_conditional_scenarios(model, crisis_intensity, num_samples=100, seq_length=252):
    """Generate synthetic scenarios with specified crisis intensity"""
    # Generate noise
    Z_mb = np.zeros((num_samples, seq_length, model.z_dim))
    T_mb = [seq_length] * num_samples

    for i in range(num_samples):
        Z_mb[i, :, :] = t_noise((seq_length, model.z_dim), df=5, scale=0.6)

    # Create crisis intensity tensor
    crisis_tensor = np.full((num_samples, 1), crisis_intensity, dtype=np.float32)

    Z_mb = tf.constant(Z_mb, dtype=tf.float32)
    crisis_tensor = tf.constant(crisis_tensor, dtype=tf.float32)

    # Generate synthetic latent representations
    H_fake = model.generator_model(Z_mb, crisis_tensor)
    H_hat_supervise = model.supervisor_model(H_fake)

    # Convert back to return space
    X_fake = model.recovery_model(H_hat_supervise)

    return X_fake.numpy()

# Example usage:
print("Generating crisis scenarios...")

# Generate calm market scenarios (low crisis intensity)
calm_scenarios = generate_conditional_scenarios(trained_conditional_model, 0.1, 50, seq_len)
print("Calm scenarios shape:", calm_scenarios.shape)

# Generate crisis scenarios (high crisis intensity)
crisis_scenarios = generate_conditional_scenarios(trained_conditional_model, 0.9, 50, seq_len)
print("Crisis scenarios shape:", crisis_scenarios.shape)

# Compare statistics
print("\nCalm scenarios statistics:")
print("Mean:", np.mean(calm_scenarios, axis=(0,1)))
print("Std:", np.std(calm_scenarios, axis=(0,1)))

print("\nCrisis scenarios statistics:")
print("Mean:", np.mean(crisis_scenarios, axis=(0,1)))
print("Std:", np.std(crisis_scenarios, axis=(0,1)))
